In [1]:
from tools import *

### Model 6: Minimize Inbound and Outbound Logistics Constrained on Customer's Demand and Factory Capacity and Sufficient Supply (Multiple Ports) (Lossy Production) (Multiple Products) (Variable Product Capacity)

Variable production means that each product will take a variable amount at one factory. Then the capacity is in fact the total production time. This requires some changes to the capacity matrix.

#### Input to the Model: 

1. Number of factories and customers and ports and products
2. Outbound cost (times the number of products)
3. Inbound cost (each port to each factory) (times the number of products)
4. Demands (times the number of products)
5. Factory Capacity (Variable)
6. Loss Percentage / Efficiency (Variable)

In [2]:
0 0 | 0.75 0.75 0.75 | 0 0 |  0    0    0   || 0 0 0 0 | 0.63 0.63 0.63 | 0 0 |  0   0   0
0 0 |  0    0    0   | 0 0 | 0.43 0.43 0.43 || 0 0 0 0 |  0    0    0   | 0 0 | .59 .59 .59

SyntaxError: invalid syntax (<ipython-input-2-f1edd574916c>, line 1)

In [34]:
no_I, no_J, no_K, no_P = 4, 5, 3, 2 # Number of factories, number of customers, number of ports

ibik = np.random.rand(no_P, no_I, no_K) # Inbound Cost
obij = np.random.rand(no_P, no_I, no_J) # Outbound Cost
wj = np.random.rand(no_P, no_J) # Demands vector
ki = np.random.rand(no_I) # Factory Capacity
ei = np.random.rand(no_P, no_I) # Efficiency If by-product: row same, by-factory: column same\
ti = np.random.rand(no_P, no_I)

while np.sum(wj) >= np.sum(ki):
    wj = np.random.rand(no_P, no_J) # Redo Demands vector
    ki = np.random.rand(no_I) # Redo Factory Capacity

In [35]:
def generate_objective_vector(no_I, no_J, no_K, no_P, ibik, obij):
    c = np.concatenate([ibik.reshape(no_P, no_I, no_K), 
                        obij.reshape(no_P, no_I, no_J)], axis = 2).flatten()
    return c

In [36]:
def generate_demand_matrix(no_I, no_J, no_K, no_P):
    
    sub_block = np.tile(np.hstack([np.zeros((no_J, no_K)), np.eye(no_J)]), reps = no_I)
    
    Wijk = block_diag(*[sub_block]*no_P)
    
    return Wijk

In [37]:
def generate_capacity_matrix(no_I, no_J, no_K, no_P, ti):

    product_block_list = []
    for p in range(no_P):
        
        factory_block_list = []
        port_block = np.zeros((no_I, no_K))
        for i in range(no_I):
            customer_column = np.zeros((no_I, 1))
            customer_column[i] = ti[p][i]
            customer_block = np.repeat(customer_column, repeats = no_J, axis = 1)
            factory_block_list.append(np.hstack([port_block, customer_block]))
            
        product_block_list += factory_block_list

    Ki = np.concatenate(product_block_list, axis = 1)
    
    return Ki

In [38]:
def generate_supply_matrix_with_efficiency(no_I, no_J, no_K, no_P, ei):

    left = np.tile(-ei.reshape(no_P*no_I, 1), reps = no_K)

    right = np.ones((no_P*no_I, no_J))

    Si = block_diag(*np.hstack([left, right]))
    
    return Si

In [39]:
## Assume non-trivial
Wijk = generate_demand_matrix(no_I, no_J, no_K, no_P) # Demand Constraint Matrix
Ki = generate_capacity_matrix(no_I, no_J, no_K, no_P, ti) # Capacity Constraint Matrix

# Sufficient Supply Matrix
Si = generate_supply_matrix_with_efficiency(no_I, no_J, no_K, no_P, ei)

In [41]:
## Standard form of our model

# New cost vector
c = generate_objective_vector(no_I, no_J, no_K, no_P, ibik, obij)

# Upper Bound
A_ub = np.vstack([-Wijk, Ki, Si])
b_ub = np.hstack([-wj.flatten(), ki.flatten(), np.zeros(no_I*no_P)])

prog = linprog(c, A_ub = A_ub, b_ub = b_ub) # Bigger than or equal constraints

result = prog.x.reshape(no_P, no_I, no_J+no_K)

In [46]:
### Check demand, note that the axis = 1 the array is 3d
aae(np.sum(result[:, :, no_K:], axis = 1), wj)

### Check weighted inbound volume vs outbound volume
aae(np.sum(result[:, :, :no_K], axis = 2)*ei, np.sum(result[:, :, no_K:], axis = 2))

In [51]:
def optimize_logistics_6(no_I, no_J, no_K, ibik, obij, wj, ki, ei, ti):
    
    assert np.sum(ki) >= np.sum(wj), 'More Demand than Capacity. Program is Impossible'

    ### Standard form of our model
    ## Assume non-trivial
    Wijk = generate_demand_matrix(no_I, no_J, no_K, no_P) # Demand Constraint Matrix
    Ki = generate_capacity_matrix(no_I, no_J, no_K, no_P, ti) # Capacity Constraint Matrix

    # Sufficient Supply Matrix
    Si = generate_supply_matrix_with_efficiency(no_I, no_J, no_K, no_P, ei)

    # New cost vector
    c = generate_objective_vector(no_I, no_J, no_K, no_P, ibik, obij)

    # Upper Bound
    A_ub = np.vstack([-Wijk, Ki, Si])
    b_ub = np.hstack([-wj.flatten(), ki.flatten(), np.zeros(no_I*no_P)])

    prog = linprog(c, A_ub = A_ub, b_ub = b_ub) # Bigger than or equal constraints

    result = prog.x.reshape(no_P, no_I, no_J+no_K)
    
    ### Check demand, note that the axis = 1 the array is 3d
    aae(np.sum(result[:, :, no_K:], axis = 1), wj)

    ### Check weighted inbound volume vs outbound volume
    aae(np.sum(result[:, :, :no_K], axis = 2)*ei, np.sum(result[:, :, no_K:], axis = 2))
    
    return prog

In [57]:
prog = optimize_logistics_6(no_I, no_J, no_K, ibik, obij, wj, ki, ei, ti)